In [32]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys

In [33]:
train_file_path = 'lab2_train.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')

test_file_path = 'lab2_test.csv'

test_data = pd.read_csv(test_file_path, delimiter=',')
print(train_data)
print(train_data.shape[0])
# Convert Timestamp column to datetime
#train_data['Timestamp'] = pd.to_datetime(train_data['Timestamp'], format='%d/%m/%Y %I:%M:%S %p')
#test_data['Timestamp'] =  pd.to_datetime(test_data['Timestamp'], format='%d/%m/%Y %I:%M:%S %p')

       user_from_id  user_to_id  is_like is_match
0              1136        3141    False    False
1              2424        3174    False    False
2              1300        3590    False    False
3               800        2736    False    False
4               883         437    False    False
...             ...         ...      ...      ...
76387          2376        3057    False    False
76388          1163         933    False    False
76389          2770        3324    False    False
76390           879         785    False    False
76391           291         470    False    False

[76392 rows x 4 columns]
76392


In [34]:
class HashFunction:
    """
    This HashFunction class can be used to create an unique hash given an alpha and beta.
    """
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def hashf(self, x, n):
        """
        Returns a hash given integers x and n.
        :param x: The value to be hashed
        :param n: The number of unique ids of all sets
        :return: The hashed value x given alpha and beta
        """
        
        hash_value = 0
        hash_value =  (self.alpha * x + self.beta) % n
        return hash_value

In [35]:
def compute_signature(hashes: list[HashFunction], ids: list[set[int]]):
    """
    This function will calculate the MinHash signature matrix from our sets of ids
    using the list of hash functions (hashes)
    :param hashes: The list of hash functions of arbitrary length
    :param ids: The list of sets of ids
    :return: The MinHash signature matrix for the given sets of ids
    """
    
    result = np.full((len(hashes), len(ids)), sys.maxsize)
    space = set().union(*ids)
    sorted_space = sorted(space)
    for p, ID in enumerate(sorted_space):
        arr = []
        for k, has in enumerate(hashes):
            arr.append(hashes[k].hashf(p, len(sorted_space)))
        for j in range(len(ids)):
            if (ID in ids[j]):
                for k, has in enumerate(hashes):
                    result[k,j] = min(arr[k], result[k,j])
    return result





    
    # START ANSWER
    # END ANSWER
    
    return result


preprocessing

In [36]:
#Remove the is_match column since that is not considered
train_data = train_data.drop('is_match', axis=1)

#sort the data by user_from_id
sorted_train_data = train_data.sort_values('user_from_id')

#filter the data by is_like = True, since we want to know for each user which users they have liked.
#The point point of a reccomender system is to recommend things and not to dissuade people from things.
filtered_sorted_train_data = sorted_train_data[sorted_train_data['is_like'] == True]

#group the data by user_from_id to create the sets of liked users for each user. 
#the aggregation created an outer list around the sets for some reason so I had to flatten it.
grouped_train_data = filtered_sorted_train_data.groupby('user_from_id')
set_grouped_train_data = grouped_train_data.agg({'user_to_id': lambda x: set(x)})
list_set_grouped_train_data = set_grouped_train_data.values.flatten().tolist()

#a print to show what the data looks like.
print(set_grouped_train_data)



               user_to_id
user_from_id             
1             {837, 3046}
2                  {3058}
3                  {1179}
4             {448, 3058}
5                    {15}
...                   ...
3682               {3185}
3688               {3238}
3689               {3366}
3707               {3444}
3714               {3568}

[2107 rows x 1 columns]


In [37]:
def print_stuff(res, hash_amount, similarity_ratio, neighbours_amount):
    for i in range(res.shape[1]):  # generate pairs
        neighbours = []
        for j in range(i + 1,res.shape[1]): 
            same = 0
            
            i_array = []
            j_array = []
            for k in range(hash_amount):
                if (len(neighbours) < neighbours_amount):
                    neighbours.append(res[k][i])
                    
                i_array.append(res[k][i])
                j_array.append(res[k][j])
                if (res[k][i] == res[k][j]):
                    same += 1
            if (same/hash_amount >= similarity_ratio and same/hash_amount < 1):
                if (len(neighbours) < neighbours_amount):
                    neighbours.append(res[k][i])
                print(i+1)
                print(j+1)
                print(set_grouped_train_data['user_to_id'].values[i])
                print(set_grouped_train_data['user_to_id'].values[j])
                print(i_array)
                print(j_array)
                print(" ")
            else:
                pass
        print(neighbours)

In [38]:
hashes = []
#amount of hash functions
hash_amount = 10
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[352, 695, 180, 999, 97, 48, 993, 514, 1117, 502]
[715, 1572, 180, 1519, 141, 48, 1534, 514, 1117, 738]
 
1
1937
{837, 3046}
{837}
[352, 695, 180, 999, 97, 48, 993, 514, 1117, 502]
[352, 695, 1457, 999, 97, 720, 993, 615, 1427, 502]
 
1
1982
{837, 3046}
{837}
[352, 695, 180, 999, 97, 48, 993, 514, 1117, 502]
[352, 695, 1457, 999, 97, 720, 993, 615, 1427, 502]
 
1
1983
{837, 3046}
{1869, 837}
[352, 695, 180, 999, 97, 48, 993, 514, 1117, 502]
[352, 695, 590, 546, 97, 360, 978, 378, 1139, 502]
 
[352, 695, 180, 999, 97]
2
4
{3058}
{448, 3058}
[1492, 1145, 1265, 1005, 649, 1290, 21, 279, 176, 1135]
[1411, 1128, 1265, 193, 649, 396, 21, 279, 176, 819]
 
[1492, 1145, 1265, 1005, 649]
3
2029
{1179}
{1483, 1820, 1179}
[223, 689, 185, 612, 340, 1509, 528, 96, 1034, 1495]
[223, 434, 185, 381, 340, 36, 396, 96, 548, 169]
 
[223, 689, 185, 612, 340]
4
932
{448, 3058}
{3058}
[1411, 1128, 1265, 193, 649, 396, 21, 279, 176, 819]
[1492, 1145, 1265, 1005, 649, 1290, 21, 279, 17

KeyboardInterrupt: 

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 20
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[115, 39, 1234, 290, 25, 849, 842, 105, 402, 1420, 512, 246, 1472, 1029, 496, 224, 59, 390, 418, 404]
[839, 223, 1234, 290, 25, 1109, 1529, 917, 1379, 1420, 875, 1382, 1472, 1029, 802, 459, 1313, 390, 418, 1295]
 
1
1937
{837, 3046}
{837}
[115, 39, 1234, 290, 25, 849, 842, 105, 402, 1420, 512, 246, 1472, 1029, 496, 224, 59, 390, 418, 404]
[115, 39, 1243, 1590, 1604, 849, 842, 105, 402, 1567, 512, 246, 1644, 1308, 496, 224, 59, 1666, 457, 404]
 
1
1982
{837, 3046}
{837}
[115, 39, 1234, 290, 25, 849, 842, 105, 402, 1420, 512, 246, 1472, 1029, 496, 224, 59, 390, 418, 404]
[115, 39, 1243, 1590, 1604, 849, 842, 105, 402, 1567, 512, 246, 1644, 1308, 496, 224, 59, 1666, 457, 404]
 
1
1983
{837, 3046}
{1869, 837}
[115, 39, 1234, 290, 25, 849, 842, 105, 402, 1420, 512, 246, 1472, 1029, 496, 224, 59, 390, 418, 404]
[115, 39, 1243, 162, 941, 849, 842, 105, 402, 208, 512, 123, 1308, 366, 496, 167, 59, 7, 457, 404]
 
2
60
{3058}
{1840}
[1594, 330, 199, 1575, 1658, 852, 305,

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 50
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data, neighbours_amount)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[1132, 120, 305, 974, 722, 170, 677, 431, 656, 695, 350, 213, 514, 125, 1390, 795, 842, 37, 1066, 579, 886, 196, 22, 334, 614, 831, 994, 1165, 487, 155, 935, 19, 296, 529, 406, 490, 76, 222, 240, 373, 859, 210, 568, 1168, 294, 775, 278, 161, 1329, 98]
[1132, 120, 305, 1539, 722, 170, 677, 1365, 1096, 1454, 350, 213, 514, 875, 1390, 1607, 1263, 37, 1614, 1460, 886, 1315, 271, 1514, 770, 1051, 994, 1165, 487, 155, 1582, 19, 1333, 1447, 790, 1592, 76, 342, 536, 373, 859, 729, 568, 1168, 1510, 775, 278, 520, 1329, 818]
 
1
1937
{837, 3046}
{837}
[1132, 120, 305, 974, 722, 170, 677, 431, 656, 695, 350, 213, 514, 125, 1390, 795, 842, 37, 1066, 579, 886, 196, 22, 334, 614, 831, 994, 1165, 487, 155, 935, 19, 296, 529, 406, 490, 76, 222, 240, 373, 859, 210, 568, 1168, 294, 775, 278, 161, 1329, 98]
[1472, 791, 1598, 974, 1617, 1296, 1030, 431, 656, 695, 1639, 861, 767, 125, 1682, 795, 842, 800, 1066, 579, 1647, 196, 22, 334, 614, 831, 1030, 1207, 1566, 1019, 935, 738, 29

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 100
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data, neighbours_amount)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)


1
936
{837, 3046}
{3046}
[981, 1375, 331, 279, 880, 6, 485, 119, 788, 1061, 1253, 139, 455, 30, 320, 1211, 62, 1095, 174, 384, 1015, 63, 1105, 63, 954, 29, 549, 1083, 975, 1498, 766, 709, 37, 384, 550, 732, 815, 608, 165, 885, 1298, 1182, 116, 585, 952, 462, 704, 684, 215, 758, 578, 757, 34, 825, 162, 1222, 783, 89, 1387, 1412, 381, 526, 441, 1415, 1483, 821, 155, 446, 689, 753, 539, 754, 630, 318, 1395, 71, 988, 919, 530, 714, 1185, 1129, 85, 333, 167, 588, 733, 287, 1068, 380, 457, 168, 893, 1003, 35, 517, 735, 128, 649, 1311]
[981, 1375, 729, 998, 880, 610, 485, 907, 1033, 1230, 1253, 139, 984, 196, 320, 1352, 622, 1095, 424, 794, 1147, 63, 1105, 1161, 954, 29, 549, 1252, 975, 1498, 766, 1005, 1489, 1230, 955, 1332, 1395, 608, 165, 1264, 1305, 1182, 116, 585, 952, 462, 1314, 684, 683, 912, 578, 886, 34, 1135, 348, 1222, 783, 647, 1553, 1700, 381, 1137, 841, 1415, 1646, 1408, 155, 753, 689, 753, 539, 1110, 1079, 856, 1398, 774, 1522, 1007, 1581, 1551, 1185, 1129, 648, 620, 958, 1125,